#### Implementation of Comprehensiveness

### Evaluating Faithfulness on our model:

In [1]:
import argparse
from pprint import pprint

import evaluate
import numpy as np
import pandas as pd
import torch
from torch import nn
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    EvalPrediction,
    OPTForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import wandb

MODEL = "facebook/opt-350m"
MAX_POSITION_EMBEDDINGS = 2048

from dataclasses import dataclass


/h/u11/c0/00/ammcourt/miniconda3/envs/csc413/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [3]:
CHECKPOINT_DIR = "OPT-350m-mimic-full"
VAL_DATASET_PATH = "data/val_9.csv"
CODE_PATH = "data/icd9_codes.csv"

In [4]:
# Load dataset
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True, device=device)

code_labels = pd.read_csv("data/icd9_codes.csv")
dataset = load_dataset("csv", data_files=VAL_DATASET_PATH)

# Create class dictionaries
classes = [class_ for class_ in code_labels["icd_code"] if class_]
class2id = {class_: id for id, class_ in enumerate(classes)}
id2class = {id: class_ for class_, id in class2id.items()}


def multi_labels_to_ids(labels: list[str]) -> list[float]:
    ids = [0.0] * len(class2id)  # BCELoss requires float as target type
    for label in labels:
        ids[class2id[label]] = 1.0
    return ids


def preprocess_function(example):
    result = tokenizer(
        example["text"], truncation=True, max_length=MAX_POSITION_EMBEDDINGS
    )
    result["labels"] = [multi_labels_to_ids(eval(label)) for label in example["labels"]]
    return result



In [5]:
config, unused_kwargs = AutoConfig.from_pretrained(
    MODEL,
    num_labels=len(classes),
    id2label=id2class,
    label2id=class2id,
    problem_type="multi_label_classification",
    return_unused_kwargs=True,
)

if unused_kwargs:
    print(f"Unused kwargs: {unused_kwargs}")

model = OPTForSequenceClassification.from_pretrained(
    MODEL,
    config=config,
).to(device)

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.load_adapter(CHECKPOINT_DIR)

In [7]:
untokenized_dataset = load_dataset("csv", data_files=VAL_DATASET_PATH)

print(untokenized_dataset['train'][0])

{'text': "Sex:   M\n \nService: SURGERY\n \nAllergies: \nGrass ___, Standard / Lactose\n \n ___.\n \nChief Complaint:\nright popliteal aneurysm\n \nMajor Surgical or Invasive Procedure:\n___: popliteal artery stent graft\n\n \nHistory of Present Illness:\nMr. ___ has a fairly focal aneurysm in the\nmid right popliteal artery and is mostly full of thrombus and\nmeasures 3.1 cm.  It is patent and does have palpable pedal\npulse distally.  He has need of upcoming ankle surgery as well.\nHe has a past medical history notable for breast cancer status\npost mastectomy and chemotherapy/radiation therapy with duodenal\nulcer, pseudogout, depression, hypothyroidism, microvascular\ncerebrovascular disease, hyperlipidemia, and COPD.  He had vein\nmapping performed today which shows the lesser saphenous veins\nto be small and noncompressible bilaterally.  He has the\nthrombosis of the right greater saphenous at the level of the\nknee.  The left greater saphenous is adequate as are both\nbasilic an

In [8]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from lime.lime_text import IndexedString
import numpy as np
import torch.nn.functional as F
from time import time


explainer = LimeTextExplainer(class_names=classes, bow=False)

def predictor_opt(texts):
    print(len(texts))
    tk = tokenizer(texts, return_tensors="pt",truncation=True, padding=True, max_length=MAX_POSITION_EMBEDDINGS).to(device)
    outputs = model(**tk)
    tensor_logits = outputs[0]
    probas = tensor_logits.sigmoid().detach().cpu().numpy()
    del tk, tensor_logits
    # probas = F.sigmoid(tensor_logits).detach().cpu().numpy()
    return probas


def predictor_model(texts, model, tokenizer):
    print(len(texts))
    tk = tokenizer(texts, return_tensors="pt",truncation=True, padding=True, max_length=MAX_POSITION_EMBEDDINGS).to(device)
    outputs = model(**tk)
    tensor_logits = outputs[0]
    probas = tensor_logits.sigmoid().detach().cpu().numpy()
    del tk, tensor_logits
    # probas = F.sigmoid(tensor_logits).detach().cpu().numpy()
    return probas

The instances are formatted as a list of strings, where each string is one word used by lime. The rationales mask is a list of indices, where the first list refers to the index of the sample the label corresponds to and the second list is the index of string used in that rationale.

### Test code for faithfulness calculation

In [9]:
# create a test with 10 instances for faithfulness evaluation
# from transformers import AutoTokenizer

# get the lime evaluations of each instance
import faithfulness
# this reimports the library for easy testing in the notebook
import importlib
importlib.reload(faithfulness)

samples_start = 0
samples_end = 50

instances = untokenized_dataset["train"][samples_start:samples_end]["text"]
print(len(instances))

# print(instances)
explainer = LimeTextExplainer(class_names=classes, bow=False)

indexed_text, index_array_rationalle = faithfulness.lime_create_index_arrays(instances, predictor_opt, explainer)
# print(indexed_text)
# print(index_array_rationalle)

# # remove the rationale words
rationalle_removed = faithfulness.remove_rationale_words(indexed_text, index_array_rationalle)
others_removed = faithfulness.remove_other_words(indexed_text, index_array_rationalle)

# rationalle_removed = rationalle_removed + rationalle_removed + rationalle_removed + rationalle_removed + rationalle_removed
# others_removed = others_removed + others_removed + others_removed + others_removed + others_removed 
# instances = instances + instances + instances + instances + instances

# print(rationalle_removed)

# print(len(rationalle_removed))
# print(len(others_removed))

# the extra list is needed since the function expects a list of instances each coming from a different interpretability method
# testing multi input by duplicating the arrays, don't actually do this
ind, faith = faithfulness.calculate_faithfulness(instances, [rationalle_removed, rationalle_removed], [others_removed, others_removed], model, tokenizer, predictor_model)
print(ind)
print(faith)

10
["Sex:   M\n \nService: SURGERY\n \nAllergies: \nGrass ___, Standard / Lactose\n \n ___.\n \nChief Complaint:\nright popliteal aneurysm\n \nMajor Surgical or Invasive Procedure:\n___: popliteal artery stent graft\n\n \nHistory of Present Illness:\nMr. ___ has a fairly focal aneurysm in the\nmid right popliteal artery and is mostly full of thrombus and\nmeasures 3.1 cm.  It is patent and does have palpable pedal\npulse distally.  He has need of upcoming ankle surgery as well.\nHe has a past medical history notable for breast cancer status\npost mastectomy and chemotherapy/radiation therapy with duodenal\nulcer, pseudogout, depression, hypothyroidism, microvascular\ncerebrovascular disease, hyperlipidemia, and COPD.  He had vein\nmapping performed today which shows the lesser saphenous veins\nto be small and noncompressible bilaterally.  He has the\nthrombosis of the right greater saphenous at the level of the\nknee.  The left greater saphenous is adequate as are both\nbasilic and cep

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.00 GiB. GPU 0 has a total capacity of 15.72 GiB of which 1.29 GiB is free. Including non-PyTorch memory, this process has 14.43 GiB memory in use. Of the allocated memory 9.37 GiB is allocated by PyTorch, and 4.86 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)